### Import modules

In [1]:
import os
import pandas as pd

os.chdir("../../")

from datasets import load_dataset
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA
from langchain_community.document_loaders import HuggingFaceDatasetLoader
from langchain_huggingface import HuggingFaceEmbeddings
from dotenv import load_dotenv

In [2]:
from src.rag_pipeline import chunk_by_recursive_split, RAGSystem
from src.env_loader import load_api_keys
from src.ragas.ragas_pipeline import run_ragas_evaluation
from src import display_df

#### Load API keys

In [3]:
openai_api_key = load_api_keys("OPENAI_API_KEY")

### Rag Setup

#### Initialize embeddings from huggingface - [all-mpnet-base-v2](https://huggingface.co/sentence-transformers/all-mpnet-base-v2)

In [4]:
embeddings=HuggingFaceEmbeddings(model_name='sentence-transformers/all-mpnet-base-v2')
# embeddings = OpenAIEmbeddings(api_key=openai_api_key, model='text-embedding-ada-002')

#### Initialize RAG system with ensemble_retriever with BM25 retriever

In [5]:

rag_system_ensemble = RAGSystem(
  model_name = "gpt-4o",
  existing_vectorstore = False,
  # use_ensemble_retriever = True,
  embeddings=embeddings,
  clear_store=True
)

In [6]:
rag_system_ensemble.initialize()

--Split 1000 documents into 5030 chunks.--


### Check the RAG system
TODO - Write a test to check if RAG system is working properly - asserts for the output

In [7]:
question = "Who was one of Putin's harshest critics?"
result = rag_system_ensemble.rag_chain.invoke(question)
result

{'question': "Who was one of Putin's harshest critics?",
 'answer': "One of Putin's harshest critics was Boris Nemtsov.",
 'contexts': ['be heading an opposition party and do what I\'m doing." Opinion: The complicated life and tragic death of Boris Nemtsov . Critics of Putin have in the past suffered miserable fates. Last year, a Moscow court sentenced five men to prison for the 2006 killing of Russian journalist and fierce Kremlin critic Anna Politkovskaya. Business magnate Mikhail Khodorkovsky accused Putin of corruption and spent 10 years in prison and labor camps. Late last year, Kremlin critic Alexey Navalny was found guilty of fraud in a politically charged trial. Russia\'s official news agency reported Monday that a request by Navalny to attend Nemtsov\'s funeral had been denied. And before his death, Nemtsov had been arrested several times for speaking against Putin\'s government. Kasparov, chairman of the Human Rights Foundation\'s International Council, suggested the killing 

## RAGAS Pipeline testing the rag_chain

In [8]:
rag_results = run_ragas_evaluation(
  rag_chain=rag_system_ensemble.rag_chain,
  save_results=True,
  experiment_name="embedding_model_allmpnetv2"
)


--LOADING EVALUATION DATA--
--EVALUATING LOCALLY--
--GETTING CONTEXT AND ANSWERS--


Evaluating:   0%|          | 0/80 [00:00<?, ?it/s]

--EVALUATION COMPLETE--
--RESULTS SAVED--
